In [ ]:
import babelnet as bn
from babelnet import BabelSynsetID, Language
from babelnet.data.relation import BabelPointer

In [2]:
bn.version()

5.0

In [4]:
from zerorpc import TimeoutExpired, LostRemote

In [ ]:
s1 = bn.get_synset(BabelSynsetID('bn:00012605n')) # watchband
s2 = bn.get_synset(BabelSynsetID('bn:00019887n')) # clock

In [ ]:
t1 = bn.get_synset(BabelSynsetID('bn:02617074n')) # console (videogiochi)
t2 = bn.get_synset(BabelSynsetID('bn:15625318n')) # Playstation 4

In [6]:
r1 = bn.get_synset(BabelSynsetID('bn:00008350n')) # braccialetto
r2 = bn.get_synset(BabelSynsetID('bn:00023240n')) # casa di campagna

In [9]:
import time
import sys
sys.path.append('../')
from utils.utils import get_current_logfile_number

In [10]:
# Ricerca stile BFS
# Tempi di computazione eccessivi oltre il secondo livello di profondità dal nodo sorgente, >5/10 min,
# crescita esponenziale (forse tempo di risposta del server troppo elevato, ma comunque metodo inefficente)
logfname = get_current_logfile_number('search_path')

timestamp_n, timestamp_step = 1, 1
max_length, n = 8, 0
visited = set()
q = [(r1,n)]
target = r2.id

start_t = time.time()
with open(logfname, 'w') as logfile:
    while q and n < max_length:
        synset, n = q.pop(0)
        if n == timestamp_n:
            timestamp_n += timestamp_step
            end_t = time.time()
            min, sec = divmod(end_t - start_t, 60)
            print(f'{n}: {int(min)}m,{int(sec)}s')

        try:
            edges = synset.outgoing_edges(BabelPointer.ANY_HYPERNYM, BabelPointer.ANY_HYPONYM)
        except LostRemote as lr:
            logfile.write(f'LostRemote,{synset.id}\n')
            edges = []

        for edge in edges:
            if edge.id_target == target:
                print(f'Reached target at depth {n+1}')
                logfile.write(f'Reached target at depth {n+1}')
                end_t = time.time()
                min, sec = divmod(end_t - start_t, 60)
                print(f'Total time: {int(min)}m,{int(sec)}s')
                n = max_length+1
                break

            if edge.id_target not in visited:
                try:
                    visited.add(edge.id_target)
                    q.append((bn.get_synset(edge.id_target), n+1))
                except TimeoutExpired as te:
                    logfile.write(f'TimeoutExpired,{synset.id}\n')
                except LostRemote as lr:
                    logfile.write(f'LostRemote,{synset.id}\n')

        if n == max_length: print('Reached max length.')
    if q == []: print('Queue empty')


Queue empty
